In [ ]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# train_df = pd.read_csv('data/course/train.csv', encoding='utf-8')
# test_df = pd.read_csv('data/course/test.csv', encoding='utf-8')
# kg_df = pd.read_csv('data/course/rel_user_course.csv', encoding='utf-8')
# concat_df = pd.concat([train_df, test_df, kg_df])
# print(train_df.shape, test_df.shape, kg_df.shape, concat_df.shape)

# le = LabelEncoder()
# concat_df['user_id'] = le.fit_transform(concat_df['user'])
# user_dict = dict(zip(le.classes_, le.transform(le.classes_)))
# concat_df['course_id'] = le.fit_transform(concat_df['course'])
# course_dict = dict(zip(le.classes_, le.transform(le.classes_)))

# train_df['user'] = train_df['user'].map(user_dict)
# train_df['course'] = train_df['course'].map(course_dict)
# test_df['user'] = test_df['user'].map(user_dict)
# test_df['course'] = test_df['course'].map(course_dict)
# kg_df['user'] = kg_df['user'].map(user_dict)
# kg_df['course'] = kg_df['course'].map(course_dict)

# fw = open('data/course/raw/user_list.txt', 'w', encoding='utf-8')
# fw.write('org_id remap_id'+'\n')
# for k,v in user_dict.items():
#     fw.write(k + ' ' + str(v) + '\n')
# fw.close()

# fw = open('data/course/raw/item_list.txt', 'w', encoding='utf-8')
# fw.write('org_id remap_id'+'\n')
# for k,v in course_dict.items():
#     fw.write(k + ' ' + str(v) + '\n')
# fw.close()

# grouped_df = train_df.groupby('user')['course'].apply(list)
# with open('data/course/raw/train.txt', 'w') as f:
#     for index, row in grouped_df.iteritems():
#         print(index, type(index), row, type(row))
#         row_str = " ".join(map(str, row))
#         f.write(f"{index} {row_str}\n")

# grouped_df = test_df.groupby('user')['course'].apply(list)
# with open('data/course/raw/test.txt', 'w') as f:
#     for index, row in grouped_df.iteritems():
#         print(index, type(index), row, type(row))
#         row_str = " ".join(map(str, row))
#         f.write(f"{index} {row_str}\n")

# import json
# import pandas as pd

# csv_file = 'data/course/course.csv'
# fr = open('data/course/course.csv', 'r', encoding='utf-8').readlines()
# data = {}
# for i in fr:
#     if 'id,name' not in i:
#         i = i.strip().split(',')
#         data[i[0]] = i[1]
# with open('data/course/course_naem.json', 'w') as json_file:
#     json.dump(data, json_file, ensure_ascii=False)

# df = pd.read_csv('study_flask/merged_recommendations_base.csv', sep=',')
# print(df['id'])

# with open('data/course/user_id.json', 'r') as file:
#     user_id = json.load(file)

# user_ids = []
# for i in range(len(df['id'])):
#     user_ids.append(user_id[str(i)])
# df['id'] = user_ids
# df.to_csv('study_flask/merged_recommendations.csv', index=False)
# print(len(user_ids))
# print(df['id'])


In [1]:
from typing import Callable, List, Optional
from typing import Optional, Union
from tqdm import tqdm
import pandas as pd
import json
import torch
from torch import Tensor
import torch.nn.functional as F
from torch.nn import Embedding, ModuleList
from torch.nn.modules.loss import _Loss
from torch_geometric.data import HeteroData, InMemoryDataset
from torch_geometric.nn.conv import LGConv
from torch_geometric.typing import Adj, OptTensor
from torch_geometric.utils import is_sparse, to_edge_index
from torch_geometric.utils import degree

class LightGCN(torch.nn.Module):
    def __init__(
        self,
        num_nodes: int,
        embedding_dim: int,
        num_layers: int,
        alpha: Optional[Union[float, Tensor]] = None,
        **kwargs,
    ):
        super().__init__()

        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        if alpha is None:
            alpha = 1. / (num_layers + 1)

        if isinstance(alpha, Tensor):
            assert alpha.size(0) == num_layers + 1
        else:
            alpha = torch.tensor([alpha] * (num_layers + 1))
        self.register_buffer('alpha', alpha)

        self.embedding = Embedding(num_nodes, embedding_dim)
        self.convs = ModuleList([LGConv(**kwargs) for _ in range(num_layers)])

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.embedding.weight)
        for conv in self.convs:
            conv.reset_parameters()

    def get_embedding(
        self,
        edge_index: Adj,
        edge_weight: OptTensor = None,
    ) -> Tensor:
        r"""Returns the embedding of nodes in the graph."""
        x = self.embedding.weight
        out = x * self.alpha[0]

        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_weight)
            out = out + x * self.alpha[i + 1]

        return out

    def forward(
        self,
        edge_index: Adj,
        edge_label_index: OptTensor = None,
        edge_weight: OptTensor = None,
    ) -> Tensor:
        if edge_label_index is None:
            if is_sparse(edge_index):
                edge_label_index, _ = to_edge_index(edge_index)
            else:
                edge_label_index = edge_index

        out = self.get_embedding(edge_index, edge_weight)

        out_src = out[edge_label_index[0]]
        out_dst = out[edge_label_index[1]]

        return (out_src * out_dst).sum(dim=-1)

    def predict_link(
        self,
        edge_index: Adj,
        edge_label_index: OptTensor = None,
        edge_weight: OptTensor = None,
        prob: bool = False,
    ) -> Tensor:
        pred = self(edge_index, edge_label_index, edge_weight).sigmoid()
        return pred if prob else pred.round()

    def recommend(
        self,
        edge_index: Adj,
        edge_weight: OptTensor = None,
        src_index: OptTensor = None,
        dst_index: OptTensor = None,
        k: int = 1,
        sorted: bool = True,
    ) -> Tensor:
        out_src = out_dst = self.get_embedding(edge_index, edge_weight)

        if src_index is not None:
            out_src = out_src[src_index]

        if dst_index is not None:
            out_dst = out_dst[dst_index]

        pred = out_src @ out_dst.t()
        top_index = pred.topk(k, dim=-1, sorted=sorted).indices

        if dst_index is not None:  # Map local top-indices to original indices.
            top_index = dst_index[top_index.view(-1)].view(*top_index.size())

        return top_index

    def link_pred_loss(self, pred: Tensor, edge_label: Tensor,
                       **kwargs) -> Tensor:
        loss_fn = torch.nn.BCEWithLogitsLoss(**kwargs)
        return loss_fn(pred, edge_label.to(pred.dtype))

    def recommendation_loss(
        self,
        pos_edge_rank: Tensor,
        neg_edge_rank: Tensor,
        node_id: Optional[Tensor] = None,
        lambda_reg: float = 1e-4,
        **kwargs,
    ) -> Tensor:
        loss_fn = BPRLoss(lambda_reg, **kwargs)
        emb = self.embedding.weight
        emb = emb if node_id is None else emb[node_id]
        return loss_fn(pos_edge_rank, neg_edge_rank, emb)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.num_nodes}, '
                f'{self.embedding_dim}, num_layers={self.num_layers})')

class BPRLoss(_Loss):
    r"""The Bayesian Personalized Ranking (BPR) loss.

    The BPR loss is a pairwise loss that encourages the prediction of an
    observed entry to be higher than its unobserved counterparts
    (see `here <https://arxiv.org/abs/2002.02126>`__).

    .. math::
        L_{\text{BPR}} = - \sum_{u=1}^{M} \sum_{i \in \mathcal{N}_u}
        \sum_{j \not\in \mathcal{N}_u} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj})
        + \lambda \vert\vert \textbf{x}^{(0)} \vert\vert^2

    where :math:`\lambda` controls the :math:`L_2` regularization strength.
    We compute the mean BPR loss for simplicity.

    Args:
        lambda_reg (float, optional): The :math:`L_2` regularization strength
            (default: 0).
        **kwargs (optional): Additional arguments of the underlying
            :class:`torch.nn.modules.loss._Loss` class.
    """
    __constants__ = ['lambda_reg']
    lambda_reg: float

    def __init__(self, lambda_reg: float = 0, **kwargs):
        super().__init__(None, None, "sum", **kwargs)
        self.lambda_reg = lambda_reg

    def forward(self, positives: Tensor, negatives: Tensor,
                parameters: Tensor = None) -> Tensor:
        r"""Compute the mean Bayesian Personalized Ranking (BPR) loss.

        .. note::

            The i-th entry in the :obj:`positives` vector and i-th entry
            in the :obj:`negatives` entry should correspond to the same
            entity (*.e.g*, user), as the BPR is a personalized ranking loss.

        Args:
            positives (Tensor): The vector of positive-pair rankings.
            negatives (Tensor): The vector of negative-pair rankings.
            parameters (Tensor, optional): The tensor of parameters which
                should be used for :math:`L_2` regularization
                (default: :obj:`None`).
        """
        log_prob = F.logsigmoid(positives - negatives).mean()

        regularization = 0
        if self.lambda_reg != 0:
            regularization = self.lambda_reg * parameters.norm(p=2).pow(2)
            regularization = regularization / positives.size(0)

        return -log_prob + regularization

class Course(InMemoryDataset):
    def __init__(
        self,
        root: str,
        transform: Optional[Callable] = None,
        pre_transform: Optional[Callable] = None,
        force_reload: bool = False,
    ) -> None:
        super().__init__(root, transform, pre_transform,
                         force_reload=force_reload)
        self.load(self.processed_paths[0], data_cls=HeteroData)

    @property
    def raw_file_names(self) -> List[str]:
        return ['user_list.txt', 'item_list.txt', 'train.txt', 'test.txt']

    @property
    def processed_file_names(self) -> str:
        return 'data.pt'

    def process(self) -> None:
        import pandas as pd

        data = HeteroData()

        # Process number of nodes for each node type:
        node_types = ['user', 'book']
        for path, node_type in zip(self.raw_paths, node_types):
            df = pd.read_csv(path, sep=' ', header=0)
            data[node_type].num_nodes = len(df)

        # Process edge information for training and testing:
        attr_names = ['edge_index', 'edge_label_index']
        for path, attr_name in zip(self.raw_paths[2:], attr_names):
            rows, cols = [], []
            with open(path, 'r') as f:
                lines = f.readlines()
            for line in lines:
                indices = line.strip().split(' ')
                for dst in indices[1:]:
                    rows.append(int(indices[0]))
                    cols.append(int(dst))
            index = torch.tensor([rows, cols])

            data['user', 'rates', 'book'][attr_name] = index
            if attr_name == 'edge_index':
                data['book', 'rated_by', 'user'][attr_name] = index.flip([0])

        if self.pre_transform is not None:
            data = self.pre_transform(data)

        self.save([data], self.processed_paths[0])


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LightGCNKG(torch.nn.Module):
    def __init__(
        self,
        num_nodes: int,
        embedding_dim: int,
        num_layers: int,
        alpha: Optional[Union[float, Tensor]] = None,
        kg_num_rels: int = 0,
        kg_embedding_dim: int = 0,
        **kwargs,
    ):
        super().__init__()

        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        if alpha is None:
            alpha = 1. / (num_layers + 1)

        if isinstance(alpha, Tensor):
            assert alpha.size(0) == num_layers + 1
        else:
            alpha = torch.tensor([alpha] * (num_layers + 1))
        self.register_buffer('alpha', alpha)

        self.embedding = Embedding(num_nodes, embedding_dim)
        self.kg_embedding = Embedding(kg_num_rels, kg_embedding_dim)
        self.convs = ModuleList([LGConv(**kwargs) for _ in range(num_layers)])

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.embedding.weight)
        torch.nn.init.xavier_uniform_(self.kg_embedding.weight)
        for conv in self.convs:
            conv.reset_parameters()

    def get_embedding(
        self,
        edge_index: Adj,
        edge_weight: OptTensor = None,
    ) -> Tensor:
        x = self.embedding.weight
        out = x * self.alpha[0]

        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_weight)
            out = out + x * self.alpha[i + 1]

        return out

    def kg_embedding_lookup(
        self,
        kg_edge_index: Adj,
        kg_edge_type: Tensor,
    ) -> Tensor:
        kg_embedding = self.kg_embedding.weight
        kg_embedding = kg_embedding[kg_edge_type]
        return kg_embedding

    def forward(
        self,
        edge_index: Adj,
        edge_label_index: OptTensor = None,
        edge_weight: OptTensor = None,
        kg_edge_index: Adj = None,
        kg_edge_type: Tensor = None,
    ) -> Tensor:
        if edge_label_index is None:
            if is_sparse(edge_index):
                edge_label_index, _ = to_edge_index(edge_index)
            else:
                edge_label_index = edge_index

        out = self.get_embedding(edge_index, edge_weight)

        if kg_edge_index is not None and kg_edge_type is not None:
            kg_embedding = self.kg_embedding_lookup(kg_edge_index, kg_edge_type)
            out = torch.cat((out, kg_embedding), dim=-1)

        out_src = out[edge_label_index[0]]
        out_dst = out[edge_label_index[1]]

        return (out_src * out_dst).sum(dim=-1)

    def predict_link(
        self,
        edge_index: Adj,
        edge_label_index: OptTensor = None,
        edge_weight: OptTensor = None,
        kg_edge_index: Adj = None,
        kg_edge_type: Tensor = None,
        prob: bool = False,
    ) -> Tensor:
        pred = self(edge_index, edge_label_index, edge_weight, kg_edge_index, kg_edge_type).sigmoid()
        return pred if prob else pred.round()

    def recommend(
        self,
        edge_index: Adj,
        edge_weight: OptTensor = None,
        src_index: OptTensor = None,
        dst_index: OptTensor = None,
        k: int = 1,
        sorted: bool = True,
        kg_edge_index: Adj = None,
        kg_edge_type: Tensor = None,
    ) -> Tensor:
        out_src = out_dst = self.get_embedding(edge_index, edge_weight)

        if kg_edge_index is not None and kg_edge_type is not None:
            kg_embedding = self.kg_embedding_lookup(kg_edge_index, kg_edge_type)
            out_src = torch.cat((out_src, kg_embedding), dim=-1)
            out_dst = torch.cat((out_dst, kg_embedding), dim=-1)

        if src_index is not None:
            out_src = out_src[src_index]

        if dst_index is not None:
            out_dst = out_dst[dst_index]

        pred = out_src @ out_dst.t()
        top_index = pred.topk(k, dim=-1, sorted=sorted).indices

        if dst_index is not None:  # Map local top-indices to original indices.
            top_index = dst_index[top_index.view(-1)].view(*top_index.size())

        return top_index

    def link_pred_loss(self, pred: Tensor, edge_label: Tensor, **kwargs) -> Tensor:
        loss_fn = torch.nn.BCEWithLogitsLoss(**kwargs)
        return loss_fn(pred, edge_label.to(pred.dtype))

    def recommendation_loss(
        self,
        pos_edge_rank: Tensor,
        neg_edge_rank: Tensor,
        node_id: Optional[Tensor] = None,
        lambda_reg: float = 1e-4,
        **kwargs,
    ) -> Tensor:
        loss_fn = BPRLoss(lambda_reg, **kwargs)
        emb = self.embedding.weight
        emb = emb if node_id is None else emb[node_id]
        return loss_fn(pos_edge_rank, neg_edge_rank, emb)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.num_nodes}, '
                f'{self.embedding_dim}, num_layers={self.num_layers})')

def dcg_k(isin_mat, k):
    dcg = isin_mat / torch.log2(torch.arange(2, k + 2))
    return dcg.sum(dim=-1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Course('data/course')
data = dataset[0]
print(data)
num_users, num_books = data['user'].num_nodes, data['book'].num_nodes
data = data.to_homogeneous().to(device)
print(data)

df_kg = pd.read_csv('data/course/kg.csv', sep='\t')
kg = []
for i in range(len(df_kg)):
    head = df_kg.iloc[i]['head']
    relation = df_kg.iloc[i]['relation']
    tail = df_kg.iloc[i]['tail']
    kg.append([head, 1, tail])
print(kg[:10])

batch_size = 8192
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCNKG(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=2,
    kg_num_rels=data.num_nodes,
    kg_embedding_dim=64
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    total_loss = total_examples = 0
    for index in tqdm(train_loader):
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(
            data.edge_index, edge_label_index,
            kg_edge_index = torch.tensor(kg),
            kg_edge_type = torch.arange(0, 199897)
            ).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    hr10 = hr20 = ndcg10 = ndcg20 = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ book_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                     data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

        # HR@10 and HR@20
        hr10 += float((isin_mat[:, :10].sum(dim=-1) > 0).sum())
        hr20 += float((isin_mat[:, :20].sum(dim=-1) > 0).sum())

        # NDCG@10 and NDCG@20
        dcg10 = dcg_k(isin_mat[:, :10], 10)
        dcg20 = dcg_k(isin_mat[:, :20], 20)
        ndcg10 += float(dcg10.sum())
        ndcg20 += float(dcg20.sum())

    return precision/total_examples, recall/total_examples, hr10/total_examples, \
        hr20/total_examples, ndcg10/total_examples, ndcg20/total_examples

for epoch in range(1, 61):
    loss = train()
    precision, recall, hr10, hr20, ndcg10, ndcg20 = test(k=20)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f},'
          f'hr@10: {hr10:.4f},'
          f'hr@20: {hr20:.4f},'
          f'ndcg@10: {ndcg10:.4f},'
          f'ndcg@20: {ndcg20:.4f}'
          )

torch.save(model, 'course_kg.bin')



HeteroData(
  user={ num_nodes=199199 },
  book={ num_nodes=698 },
  (user, rates, book)={
    edge_index=[2, 614476],
    edge_label_index=[2, 68276],
  },
  (book, rated_by, user)={ edge_index=[2, 614476] }
)
Data(edge_index=[2, 1228952], edge_label_index=[2, 68276], node_type=[199897], edge_type=[1228952])
[[55176, 1, 239], [154324, 1, 239], [46174, 1, 239], [82528, 1, 589], [40650, 1, 589], [109258, 1, 589], [74100, 1, 589], [33313, 1, 589], [78825, 1, 589], [48569, 1, 589]]


100%|██████████| 76/76 [01:40<00:00,  1.32s/it]


Epoch: 001, Loss: 0.5305,Precision@20: 0.0267, Recall@20: 0.4513,hr@10: 0.3700,hr@20: 0.4893,ndcg@10: 0.2344,ndcg@20: 0.2690


100%|██████████| 76/76 [01:33<00:00,  1.23s/it]


Epoch: 002, Loss: 0.2700,Precision@20: 0.0279, Recall@20: 0.4707,hr@10: 0.3932,hr@20: 0.5097,ndcg@10: 0.2534,ndcg@20: 0.2877


...

Epoch: 058, Loss: 0.0164,Precision@20: 0.0334, Recall@20: 0.5647,hr@10: 0.4976,hr@20: 0.6045,ndcg@10: 0.3453,ndcg@20: 0.3784


100%|██████████| 76/76 [01:15<00:00,  1.01it/s]


Epoch: 059, Loss: 0.0162,Precision@20: 0.0335, Recall@20: 0.5653,hr@10: 0.4979,hr@20: 0.6053,ndcg@10: 0.3459,ndcg@20: 0.3791


100%|██████████| 76/76 [15:01<00:00, 11.86s/it]   


Epoch: 060, Loss: 0.0157,Precision@20: 0.0334, Recall@20: 0.5647,hr@10: 0.4970,hr@20: 0.6046,ndcg@10: 0.3455,ndcg@20: 0.3788


In [3]:
class LightGCNKG(torch.nn.Module):
    def __init__(
        self,
        num_nodes: int,
        embedding_dim: int,
        num_layers: int,
        alpha: Optional[Union[float, Tensor]] = None,
        kg_num_rels: int = 0,
        kg_embedding_dim: int = 0,
        **kwargs,
    ):
        super().__init__()

        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        if alpha is None:
            alpha = 1. / (num_layers + 1)

        if isinstance(alpha, Tensor):
            assert alpha.size(0) == num_layers + 1
        else:
            alpha = torch.tensor([alpha] * (num_layers + 1))
        self.register_buffer('alpha', alpha)

        self.embedding = Embedding(num_nodes, embedding_dim)
        self.kg_embedding = Embedding(kg_num_rels, kg_embedding_dim)
        self.convs = ModuleList([LGConv(**kwargs) for _ in range(num_layers)])

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.embedding.weight)
        torch.nn.init.xavier_uniform_(self.kg_embedding.weight)
        for conv in self.convs:
            conv.reset_parameters()

    def get_embedding(
        self,
        edge_index: Adj,
        edge_weight: OptTensor = None,
    ) -> Tensor:
        x = self.embedding.weight
        out = x * self.alpha[0]

        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_weight)
            out = out + x * self.alpha[i + 1]

        return out

    def kg_embedding_lookup(
        self,
        kg_edge_index: Adj,
        kg_edge_type: Tensor,
    ) -> Tensor:
        kg_embedding = self.kg_embedding.weight
        kg_embedding = kg_embedding[kg_edge_type]
        return kg_embedding

    def forward(
        self,
        edge_index: Adj,
        edge_label_index: OptTensor = None,
        edge_weight: OptTensor = None,
        kg_edge_index: Adj = None,
        kg_edge_type: Tensor = None,
    ) -> Tensor:
        if edge_label_index is None:
            if is_sparse(edge_index):
                edge_label_index, _ = to_edge_index(edge_index)
            else:
                edge_label_index = edge_index

        out = self.get_embedding(edge_index, edge_weight)

        if kg_edge_index is not None and kg_edge_type is not None:
            kg_embedding = self.kg_embedding_lookup(kg_edge_index, kg_edge_type)
            out = torch.cat((out, kg_embedding), dim=-1)

        out_src = out[edge_label_index[0]]
        out_dst = out[edge_label_index[1]]

        return (out_src * out_dst).sum(dim=-1)

    def predict_link(
        self,
        edge_index: Adj,
        edge_label_index: OptTensor = None,
        edge_weight: OptTensor = None,
        kg_edge_index: Adj = None,
        kg_edge_type: Tensor = None,
        prob: bool = False,
    ) -> Tensor:
        pred = self(edge_index, edge_label_index, edge_weight, kg_edge_index, kg_edge_type).sigmoid()
        return pred if prob else pred.round()

    def recommend(
        self,
        edge_index: Adj,
        edge_weight: OptTensor = None,
        src_index: OptTensor = None,
        dst_index: OptTensor = None,
        k: int = 1,
        sorted: bool = True,
        kg_edge_index: Adj = None,
        kg_edge_type: Tensor = None,
    ) -> Tensor:
        out_src = out_dst = self.get_embedding(edge_index, edge_weight)

        if kg_edge_index is not None and kg_edge_type is not None:
            kg_embedding = self.kg_embedding_lookup(kg_edge_index, kg_edge_type)
            out_src = torch.cat((out_src, kg_embedding), dim=-1)
            out_dst = torch.cat((out_dst, kg_embedding), dim=-1)

        if src_index is not None:
            out_src = out_src[src_index]

        if dst_index is not None:
            out_dst = out_dst[dst_index]

        pred = out_src @ out_dst.t()
        top_index = pred.topk(k, dim=-1, sorted=sorted).indices

        if dst_index is not None:  # Map local top-indices to original indices.
            top_index = dst_index[top_index.view(-1)].view(*top_index.size())

        return top_index

    def link_pred_loss(self, pred: Tensor, edge_label: Tensor, **kwargs) -> Tensor:
        loss_fn = torch.nn.BCEWithLogitsLoss(**kwargs)
        return loss_fn(pred, edge_label.to(pred.dtype))

    def recommendation_loss(
        self,
        pos_edge_rank: Tensor,
        neg_edge_rank: Tensor,
        node_id: Optional[Tensor] = None,
        lambda_reg: float = 1e-4,
        **kwargs,
    ) -> Tensor:
        loss_fn = BPRLoss(lambda_reg, **kwargs)
        emb = self.embedding.weight
        emb = emb if node_id is None else emb[node_id]
        return loss_fn(pos_edge_rank, neg_edge_rank, emb)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.num_nodes}, '
                f'{self.embedding_dim}, num_layers={self.num_layers})')

def dcg_k(isin_mat, k):
    dcg = isin_mat / torch.log2(torch.arange(2, k + 2))
    return dcg.sum(dim=-1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Course('data/course')
data = dataset[0]
print(data)
num_users, num_books = data['user'].num_nodes, data['book'].num_nodes
data = data.to_homogeneous().to(device)
print(data)

df_kg = pd.read_csv('data/course/kg.csv', sep='\t')
kg = []
for i in range(len(df_kg)):
    head = df_kg.iloc[i]['head']
    relation = df_kg.iloc[i]['relation']
    tail = df_kg.iloc[i]['tail']
    kg.append([head, 1, tail])
print(kg[:10])

batch_size = 8192
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCNKG(
    num_nodes=data.num_nodes,
    embedding_dim=128,
    num_layers=2,
    kg_num_rels=data.num_nodes,
    kg_embedding_dim=128
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train():
    total_loss = total_examples = 0
    for index in tqdm(train_loader):
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(
            data.edge_index, edge_label_index,
            kg_edge_index = torch.tensor(kg),
            kg_edge_type = torch.arange(0, 199897)
            ).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    hr10 = hr20 = ndcg10 = ndcg20 = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ book_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                     data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

        # HR@10 and HR@20
        hr10 += float((isin_mat[:, :10].sum(dim=-1) > 0).sum())
        hr20 += float((isin_mat[:, :20].sum(dim=-1) > 0).sum())

        # NDCG@10 and NDCG@20
        dcg10 = dcg_k(isin_mat[:, :10], 10)
        dcg20 = dcg_k(isin_mat[:, :20], 20)
        ndcg10 += float(dcg10.sum())
        ndcg20 += float(dcg20.sum())

    return precision/total_examples, recall/total_examples, hr10/total_examples, \
        hr20/total_examples, ndcg10/total_examples, ndcg20/total_examples

for epoch in range(1, 201):
    loss = train()
    precision, recall, hr10, hr20, ndcg10, ndcg20 = test(k=20)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f},'
          f'hr@10: {hr10:.4f},'
          f'hr@20: {hr20:.4f},'
          f'ndcg@10: {ndcg10:.4f},'
          f'ndcg@20: {ndcg20:.4f}'
          )

torch.save(model, 'course_kg.bin')



HeteroData(
  user={ num_nodes=199199 },
  book={ num_nodes=698 },
  (user, rates, book)={
    edge_index=[2, 614476],
    edge_label_index=[2, 68276],
  },
  (book, rated_by, user)={ edge_index=[2, 614476] }
)
Data(edge_index=[2, 1228952], edge_label_index=[2, 68276], node_type=[199897], edge_type=[1228952])
[[55176, 1, 239], [154324, 1, 239], [46174, 1, 239], [82528, 1, 589], [40650, 1, 589], [109258, 1, 589], [74100, 1, 589], [33313, 1, 589], [78825, 1, 589], [48569, 1, 589]]


100%|██████████| 76/76 [05:03<00:00,  3.99s/it]


Epoch: 001, Loss: 0.6908,Precision@20: 0.0250, Recall@20: 0.4214,hr@10: 0.3554,hr@20: 0.4573,ndcg@10: 0.2144,ndcg@20: 0.2440


100%|██████████| 76/76 [04:52<00:00,  3.85s/it]


Epoch: 002, Loss: 0.6565,Precision@20: 0.0247, Recall@20: 0.4160,hr@10: 0.3524,hr@20: 0.4516,ndcg@10: 0.2118,ndcg@20: 0.2410


...

Epoch: 198, Loss: 0.0290,Precision@20: 0.0329, Recall@20: 0.5565,hr@10: 0.4861,hr@20: 0.5966,ndcg@10: 0.3327,ndcg@20: 0.3666


100%|██████████| 76/76 [03:45<00:00,  2.97s/it]


Epoch: 199, Loss: 0.0291,Precision@20: 0.0329, Recall@20: 0.5565,hr@10: 0.4862,hr@20: 0.5967,ndcg@10: 0.3329,ndcg@20: 0.3668


100%|██████████| 76/76 [03:37<00:00,  2.86s/it]


Epoch: 200, Loss: 0.0286,Precision@20: 0.0329, Recall@20: 0.5569,hr@10: 0.4865,hr@20: 0.5971,ndcg@10: 0.3331,ndcg@20: 0.3670


In [ ]:
def dcg_k(isin_mat, k):
    dcg = isin_mat / torch.log2(torch.arange(2, k + 2))
    return dcg.sum(dim=-1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Course('data/course')
data = dataset[0]
print(data)
num_users, num_books = data['user'].num_nodes, data['book'].num_nodes
data = data.to_homogeneous().to(device)
print(data)

batch_size = 8192
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=2,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    total_loss = total_examples = 0
    for index in tqdm(train_loader):
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    hr10 = hr20 = ndcg10 = ndcg20 = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ book_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                     data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

        # HR@10 and HR@20
        hr10 += float((isin_mat[:, :10].sum(dim=-1) > 0).sum())
        hr20 += float((isin_mat[:, :20].sum(dim=-1) > 0).sum())

        # NDCG@10 and NDCG@20
        dcg10 = dcg_k(isin_mat[:, :10], 10)
        dcg20 = dcg_k(isin_mat[:, :20], 20)
        ndcg10 += float(dcg10.sum())
        ndcg20 += float(dcg20.sum())

    return precision/total_examples, recall/total_examples, hr10/total_examples, \
        hr20/total_examples, ndcg10/total_examples, ndcg20/total_examples

for epoch in range(1, 60):
    loss = train()
    precision, recall, hr10, hr20, ndcg10, ndcg20 = test(k=20)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f},'
          f'hr@10: {hr10:.4f},'
          f'hr@20: {hr20:.4f},'
          f'ndcg@10: {ndcg10:.4f},'
          f'ndcg@20: {ndcg20:.4f}'
          )

torch.save(model, 'course.bin')


In [4]:
def dcg_k(isin_mat, k):
    dcg = isin_mat / torch.log2(torch.arange(2, k + 2))
    return dcg.sum(dim=-1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Course('data/course')
data = dataset[0]
print(data)
num_users, num_books = data['user'].num_nodes, data['book'].num_nodes
data = data.to_homogeneous().to(device)
print(data)

batch_size = 8192
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=128,
    num_layers=2,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train():
    total_loss = total_examples = 0
    for index in tqdm(train_loader):
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    hr10 = hr20 = ndcg10 = ndcg20 = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ book_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                     data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

        # HR@10 and HR@20
        hr10 += float((isin_mat[:, :10].sum(dim=-1) > 0).sum())
        hr20 += float((isin_mat[:, :20].sum(dim=-1) > 0).sum())

        # NDCG@10 and NDCG@20
        dcg10 = dcg_k(isin_mat[:, :10], 10)
        dcg20 = dcg_k(isin_mat[:, :20], 20)
        ndcg10 += float(dcg10.sum())
        ndcg20 += float(dcg20.sum())

    return precision/total_examples, recall/total_examples, hr10/total_examples, \
        hr20/total_examples, ndcg10/total_examples, ndcg20/total_examples

for epoch in range(1, 201):
    loss = train()
    precision, recall, hr10, hr20, ndcg10, ndcg20 = test(k=20)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f},'
          f'hr@10: {hr10:.4f},'
          f'hr@20: {hr20:.4f},'
          f'ndcg@10: {ndcg10:.4f},'
          f'ndcg@20: {ndcg20:.4f}'
          )

torch.save(model, 'course.bin')


HeteroData(
  user={ num_nodes=199199 },
  book={ num_nodes=698 },
  (user, rates, book)={
    edge_index=[2, 614476],
    edge_label_index=[2, 68276],
  },
  (book, rated_by, user)={ edge_index=[2, 614476] }
)
Data(edge_index=[2, 1228952], edge_label_index=[2, 68276], node_type=[199897], edge_type=[1228952])


100%|██████████| 76/76 [02:43<00:00,  2.15s/it]


Epoch: 001, Loss: 0.6908,Precision@20: 0.0252, Recall@20: 0.4258,hr@10: 0.3572,hr@20: 0.4616,ndcg@10: 0.2186,ndcg@20: 0.2489


100%|██████████| 76/76 [02:35<00:00,  2.05s/it]


Epoch: 002, Loss: 0.6571,Precision@20: 0.0251, Recall@20: 0.4229,hr@10: 0.3545,hr@20: 0.4588,ndcg@10: 0.2205,ndcg@20: 0.2509


...

Epoch: 198, Loss: 0.0710,Precision@20: 0.0332, Recall@20: 0.5606,hr@10: 0.4960,hr@20: 0.5996,ndcg@10: 0.3444,ndcg@20: 0.3765


100%|██████████| 76/76 [01:26<00:00,  1.13s/it]


Epoch: 199, Loss: 0.0706,Precision@20: 0.0332, Recall@20: 0.5606,hr@10: 0.4962,hr@20: 0.5996,ndcg@10: 0.3444,ndcg@20: 0.3766


100%|██████████| 76/76 [01:25<00:00,  1.13s/it]


Epoch: 200, Loss: 0.0701,Precision@20: 0.0332, Recall@20: 0.5609,hr@10: 0.4959,hr@20: 0.5999,ndcg@10: 0.3446,ndcg@20: 0.3769


KG :
    
    lr=0.001 Epoch: 060, Loss: 0.0157,Precision@20: 0.0334, Recall@20: 0.5647,hr@10: 0.4970,hr@20: 0.6046,ndcg@10: 0.3455,ndcg@20: 0.3788
    
    lr=0.0001 Epoch: 200, Loss: 0.0286,Precision@20: 0.0329, Recall@20: 0.5569,hr@10: 0.4865,hr@20: 0.5971,ndcg@10: 0.3331,ndcg@20: 0.3670

Linght:
    
    lr=0.0001 Epoch: 200, Loss: 0.0701,Precision@20: 0.0332, Recall@20: 0.5609,hr@10: 0.4959,hr@20: 0.5999,ndcg@10: 0.3446,ndcg@20: 0.3769

| Model 模型 | HR@10 | HR@20 | NDCG@10 | NDCG@20 |
| --- | --- | --- | --- | --- |
| LightGCN-KG lr=0.0001 | 0.4965 | 0.5971 | 0.3331 | 0.367 |
| LightGCN-KG lr=0.001 | 0.4970 | 0.6946 | 0.3455 | 0.3788 |
| LightGCN-only lr=0.0001 | 0.4959 | 0.5609 | 0.3446 | 0.3769 |
| LightGCN-only lr=0.001 | 0.500 | 0.601 | 0.3576 | 0.3893 |

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_nodes = 199897
num_edges = 1228952
edge_index = torch.randint(0, num_nodes, (2, num_edges))
edge_label_index = torch.randint(0, num_nodes, (2, 68276))
with_edge_weight = False
edge_weight = torch.rand(num_edges) if with_edge_weight else None

model = torch.load('course.bin')

with open('data/course/user_id.json', 'r') as file:
    user_id = json.load(file)
with open('data/course/course_id.json', 'r') as file:
    course_id = json.load(file)
with open('data/course/course_naem.json', 'r') as file:
    course_name = json.load(file)

inputs_user = 'U_10000060'
keys_with_user_id = [key for key, value in user_id.items() if value == inputs_user]
src_index = int(keys_with_user_id[0])
dst_index = torch.arange(0, 698)
out = model.recommend(edge_index, edge_weight, src_index, dst_index, k=20)
for i in out:
    print(i.item(), course_id[str(i.item())], course_name[course_id[str(i.item())]])


504 C_course-v1:TsinghuaX+40260173X+2019_T1 数字集成电路分析与设计
208 C_course-v1:KMUSTX+1193004+2019_T1 极速悟透3ds Max
677 C_course-v1:dlmu+20180906+2019_T1 航海概论
483 C_course-v1:TsinghuaX+34000312X+2019_T1 医学寄生虫学
692 C_course-v1:qhnu+20181212x+2018_T2 藏语言文字学概论
329 C_course-v1:SichuanU+306197030+2019_T1 土力学
631 C_course-v1:WellesleyX+ANTH207x+sp 人类进化导论
539 C_course-v1:TsinghuaX+80000271X+2019_T1 住宅精细化设计
479 C_course-v1:TsinghuaX+30803273X+2019_T1 中国工笔人物画赏析与创作
561 C_course-v1:TsinghuaX+80590982X+2019_T1 公共管理前沿——社会企业家精神
516 C_course-v1:TsinghuaX+60510102X+2019_T1 创办新企业
129 C_course-v1:FUDANx+SOSC120007+2019_T1 货币金融学
76 C_course-v1:CIE+CIE2016002+2019_T1 创业失败：原因与出路
133 C_course-v1:GDUT+2018120405X+2019_T1 音乐漫步
291 C_course-v1:RiceX+RELI157x+sp 宗教和嘻哈文化
553 C_course-v1:TsinghuaX+80240372X+2019_T1 数据挖掘：理论与算法
448 C_course-v1:TsinghuaX+20430064X+sp 量子力学
198 C_course-v1:JNUX+07009156X+sp 中医与诊断-学做自己的医生
254 C_course-v1:NEU+2019012201X+2019_T2 物理化学
627 C_course-v1:WUT+WUT2016001+2019_T1 网络、群体与市场


In [ ]:
import json
import pandas as pd

csv_file = 'data/course/course.csv'
fr = open('data/course/course.csv', 'r', encoding='utf-8').readlines()
data = {}
for i in fr:
    if 'id,name' not in i:
        i = i.strip().split(',')
        data[i[0]] = i[1]
with open('data/course/course_naem.json', 'w') as json_file:
    json.dump(data, json_file, ensure_ascii=False)

df = pd.read_csv('study_flask/merged_recommendations_base.csv', sep=',')
print(df['id'])

with open('data/course/user_id.json', 'r') as file:
    user_id = json.load(file)

user_ids = []
for i in range(len(df['id'])):
    user_ids.append(user_id[str(i)])
df['id'] = user_ids
df.to_csv('study_flask/merged_recommendations.csv', index=False)
print(len(user_ids))
print(df['id'])